In [ ]:
# Importing necessary libraries

!pip install cellpylib

In [ ]:
# Importing necessary libraries

import cellpylib as cpl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import seaborn as sns
import time

In [ ]:
# Defining plots and the neural network architecture uses

# Functions (by themselves)

# Model Initialisation, WITH BATCH NORMALISATION
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.bn2 = nn.BatchNorm1d(num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        return out

def create_data(data_size, programmes_considered, number_of_samples, timesteps):
  # creating data set, by randomly initialising number_of_samples times with random programmes.
  dataset = np.empty(shape=(number_of_samples, data_size), dtype=int) # each row is data_size length, with number_of_samples rows
  labels = np.empty(shape=(1, number_of_samples), dtype=int)
  #print(f"labels = ", labels)
  #print(dataset)
  for i in range(number_of_samples):

    #randomly selecting a rule number
    rule_number = np.random.choice(programmes_considered)
    #print(f"Considering rule_number = ", rule_number)
    cellular_automaton = cpl.init_random(data_size)
    cellular_automaton = cpl.evolve(cellular_automaton, timesteps=timesteps, memoize=True, apply_rule=lambda n, c, t: cpl.nks_rule(n, rule_number))
    #print(cellular_automaton[-1])
    dataset[i] = cellular_automaton[-1]
    labels[:,i] = rule_number

  return [dataset, labels]

def data_split(data):

  np.random.shuffle(data) #randomly select parts of the dataset
  #train_ratio = train_ratio # this reserves 80% for training, 20% for testing
  split_index = int(len(data) * train_ratio)

  train_data = data[:split_index]
  test_data = data[split_index:]
  #print(f"train_data = ", train_data)
  #print(f"test_data = ", test_data)

  # Separate the dataset and labels from the training and testing sets
  train_dataset, train_labels = zip(*train_data)
  test_dataset, test_labels = zip(*test_data)

  data_split = [train_dataset, train_labels, test_dataset, test_labels]
  return data_split

In [ ]:
# This code is copied over from the google drive. Now with significant alterations
from tqdm import tqdm
#@title Set System Parameters
data_size = 100 # the number of data points in each row of data
programmes_considered = np.arange(0,256,1) # the set of programmes being considered. For the 1D case it makes sense to consider all 0 to 255 programmes.
number_of_samples = 2560*2 # the number of random times the output of a programme will be calculated, given random inputs
timesteps = 100 # the number of timesteps which each programme is run for before the output is used to train the model

#laterly used parameters
num_epochs = 300  # Number of training epochs
hidden_size = 256  # Update with the desired size of the hidden layer
learning_rate = 0.001 # learning rate used later in the optimizer
batch_size = 32 # Batch size used when creating the train and test datasets. Note that 5 is likely much too low, and 32 would be more suitable for this problem.
train_ratio = 0.8 # Specifies how much of the set will be used to training vs testing

num_repeats = 3

fig, axs = plt.subplots(1, num_repeats, figsize=(5 * num_repeats, 5), sharey=True)

for repeat_idx in range(num_repeats):
    
    # Define the input size, hidden size, and number of classes
    input_size = data_size  # Update with the actual input size
    #hidden_size = 64  # Update with the desired size of the hidden layer
    num_classes = len(programmes_considered)+1  # Number of potential classes
    
    # Create an instance of the neural network
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    
    #@title Evaluating the train and test splits
    [dataset, labels] = create_data(data_size, programmes_considered, number_of_samples, timesteps)
    labels = labels[0]
    # Shifting the labels such that they are indexed from 0. Required for cross entropy to work
    labels = [x - min(labels) for x in labels]
    data = [(data_sample, label) for data_sample, label in zip(dataset, labels)]
    [train_dataset, train_labels, test_dataset, test_labels] = data_split(data)
    
    #@title Setting up Training
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Prepare your dataset and data loaders (train_loader and test_loader)
    # Create a TensorDataset from the train_dataset and train_labels
    #dataset = torch.utils.data.TensorDataset([train_dataset, train_labels])
    tensor_train_dataset = TensorDataset(Tensor(train_dataset), Tensor(train_labels))
    tensor_test_dataset = TensorDataset(Tensor(test_dataset), Tensor(test_labels))
    
    train_loader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(tensor_test_dataset, batch_size=batch_size, shuffle=True)
    #print(f"train_loader = ", train_loader)
    
    # Creating variables to track the change in error over time
    training_loss = np.empty(num_epochs)
    test_loss = np.empty(num_epochs)
    
    # Training loop
    #num_epochs = 100  # Number of training epochs
    for epoch in tqdm(range(num_epochs)):
        for data, labels in train_loader:
            #print(f"data = ", data)
            #print(f"labels = ", labels)
            labels = labels.long() #required for the calculation of CrossEntropyLoss
            #print(f"labels = ", labels)
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, labels)
    
            # monitoring test loss during training
            for data, labels in test_loader:
                labels_test = labels.long()
                outputs_test = model(data)
                loss_test = criterion(outputs_test, labels_test)
                test_loss[epoch] = loss_test.item()
    
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        # Print the loss after each epoch
        #if epoch%10==0:
        #    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        training_loss[epoch] = loss.item()
    
    # Evaluation on the training dataset (relevant for overparametrisation or otherwise deep learning systems)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, labels in train_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print(f"Accuracy on the train set: {accuracy}%")
    
    # Evaluation on the testing dataset
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, labels in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print(f"Accuracy on the test set: {accuracy}%")
    
    parameter_number = sum(p.numel() for p in model.parameters())
    epochs = np.arange(0, num_epochs, 1)
    axs[repeat_idx].plot(epochs, training_loss, test_loss)
    axs[repeat_idx].set_xlabel("Epoch")
    axs[repeat_idx].set_ylabel("Cross Entropy Loss")
    axs[repeat_idx].title.set_text(f"Run {repeat_idx+1}")
    axs[repeat_idx].set_ylim(bottom = 0)
    axs[repeat_idx].legend(["Training Loss", "Test Loss"])

#plt.tight_layout()
plt.suptitle(f'Model Performance')
# Show the subplots
plt.show()